In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from utils.roi_measures import mad, ssim
from utils.extraction import extract_patches
from utils.reconstruction import perform_voting

from model3d import Multimodel

/home/jose/joses-area/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
import nibabel as nib

from medical_data import cdr_info, nwbv_info, diff_info

file_general_pattern = 'OAS2_0{0:03}_MR{1}_{3}_OAS2_0{0:03}_MR{2}'
dataset_location = 'datasets/OASIS/OASIS2/REG/{}/{}.nii.gz'

step = (16, 16, 16)
threshold = np.int32(0.30 * np.prod(curr_patch_shape[:]))
seg_train = np.empty((0, 2, ) + curr_patch_shape)
ref_train = np.empty((0, 2, ) + curr_patch_shape)
out_train = np.empty((0, 1, ) + curr_patch_shape)
for i in range(1, 100) :
    print '{} :'.format(i),
    
    for j in range(1, 5) :
        for k in range(j+1, 5) :
            ref_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, j, k, 'halfwayto'))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, j, k, 'halfwayto') + '_brain_pve_0')
            mov_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, k, j, 'halfwayto'))
            mov_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, k, j, 'halfwayto') + '_pve_0')
            
            if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
                continue

            ######################################################################################
            volume_init = nib.load(ref_filename).get_data()
            volume_init = volume_init / volume_init.max()
            
            mask_patches = extract_patches(volume_init != 0, curr_patch_shape, step)

            useful_patches = np.sum(mask_patches, axis=(1, 2, 3)) > threshold
            
            del mask_patches
            
            ref_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_patches = ref_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(ref_prob_filename).get_data()
            
            ref_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_prob_patches = ref_prob_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            ref_train = np.vstack((np.hstack((ref_patches, ref_prob_patches)), ref_train)).astype('float32') ##
            del ref_patches, ref_prob_patches

            ######################################################################################
            volume_init = nib.load(mov_filename).get_data()
            volume_init = volume_init / volume_init.max()
            
            mov_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_patches = mov_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(ref_prob_filename).get_data()
            
            mov_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_prob_patches = mov_prob_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            seg_train = np.vstack((np.hstack((mov_prob_patches, mov_patches != 0)), seg_train)).astype('float32') ##
            out_train = np.vstack((mov_patches, out_train)).astype('float32')
            del mov_patches, mov_prob_patches
            ######################################################################################
            print '{}->{}'.format(j, k),
    print

1 : 1->2
2 : 1->2 1->3 2->3
3 :
4 : 1->2
5 : 1->2 1->3 2->3
6 :
7 : 1->3 1->4 3->4
8 : 1->2
9 : 1->2
10 : 1->2
11 :
12 : 1->2 1->3 2->3
13 : 1->2 1->3 2->3
14 : 1->2
15 :
16 : 1->2
17 : 1->3 1->4 3->4
18 : 1->3 1->4 3->4
19 :
20 : 1->2 1->3 2->3
21 : 1->2
22 : 1->2
23 : 1->2
24 :
25 :
26 : 1->2
27 : 1->2 1->3 1->4 2->3 2->4 3->4
28 : 1->2
29 : 1->2
30 : 1->2
31 : 1->2 1->3 2->3
32 : 1->2
33 :
34 : 1->2 1->3 1->4 2->3 2->4 3->4
35 : 1->2
36 : 1->3 1->4 3->4
37 : 1->2 1->3 1->4 2->3 2->4 3->4
38 :
39 : 1->2
40 : 1->2 1->3 2->3
41 : 1->2 1->3 2->3
42 : 1->2
43 : 1->2
44 : 1->2 1->3 2->3
45 : 1->2
46 : 1->2
47 : 1->2
48 : 1->2 1->3 1->4 2->3 2->4 3->4
49 : 1->2 1->3 2->3
50 : 1->2
51 : 1->2 1->3 2->3
52 : 1->2
53 : 1->2
54 : 1->2
55 : 1->2
56 : 1->2
57 : 1->2 1->3 2->3
58 : 1->2 1->3 2->3
59 :
60 : 1->2
61 : 1->2 1->3 2->3
62 : 1->2 1->3 2->3
63 : 1->2
64 : 1->2 1->3 2->3
65 :
66 : 1->2
67 : 1->2 1->3 1->4 2->3 2->4 3->4
68 : 1->2
69 : 1->2
70 : 1->2 1->3 1->4 2->3 2->4 3->4
71 : 1->2
72 :

In [2]:
curr_patch_shape = (32, 32, 32)
input_modalities = ['T1', 'seg']
output_modalities = ['Gen']
output_weights = {'Gen' : 1.0, 'concat' : 1.0}
latent_dim = 16
channels = [2, 2]
patch_shape = curr_patch_shape
scale = 0.5
a_model = Multimodel(
    input_modalities, output_modalities, output_weights, latent_dim, channels, patch_shape, scale)
a_model.build()

Latent dimensions: 16
Fuse latent representations using max
making output: Tensor("enc_T1_act9/add:0", shape=(?, 16, 32, 32, 32), dtype=float32) Tensor("dec_Gen/dec_Gen_act5/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_0_dec_Gen
making output: Tensor("enc_seg_act9/add:0", shape=(?, 16, 32, 32, 32), dtype=float32) Tensor("dec_Gen_1/dec_Gen_act5/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_1_dec_Gen
making output: Tensor("combined_em/Maximum:0", shape=(?, 16, 32, 32, 32), dtype=float32) Tensor("dec_Gen_2/dec_Gen_act5/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_2_dec_Gen
making output: em_concat Tensor("em_concat/concat:0", shape=(?, 2, ?), dtype=float32) em_concat/concat:0
all outputs:  [u'em_0_dec_Gen/add:0', u'em_1_dec_Gen/add:0', u'em_2_dec_Gen/add:0', u'em_concat/concat:0', u'em_fused/ExpandDims:0']
output dict:  {'em_concat': <function embedding_distance at 0x7ff010bd8aa0>, 'em_fused': <function embedding_distance at 0x7ff010bd8aa0>, 'em_1_dec_Gen': <function m

In [4]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

patience = 3

stopper = EarlyStopping(patience=patience)
checkpointer = ModelCheckpoint('models/ag_3d_1-100.h5', save_best_only=True, save_weights_only=True)

N = len(ref_train)
a_model.model.fit(
    [ref_train, seg_train],
    [out_train, out_train, out_train, np.empty((N, 2, 0)), np.empty((N, 1, 0))],
    validation_split=0.3, epochs=40,
    callbacks=[checkpointer, stopper])

Train on 49109 samples, validate on 21048 samples
Epoch 1/40
49109/49109 [==============================] - 1562s 32ms/step - loss: 0.1161 - em_0_dec_Gen_loss: 0.0369 - em_1_dec_Gen_loss: 0.0433 - em_2_dec_Gen_loss: 0.0349 - em_concat_loss: 0.0011 - em_fused_loss: 0.0000e+00 - val_loss: 0.1269 - val_em_0_dec_Gen_loss: 0.0418 - val_em_1_dec_Gen_loss: 0.0453 - val_em_2_dec_Gen_loss: 0.0393 - val_em_concat_loss: 5.0173e-04 - val_em_fused_loss: 0.0000e+00
Epoch 2/40
49109/49109 [==============================] - 1521s 31ms/step - loss: 0.1051 - em_0_dec_Gen_loss: 0.0342 - em_1_dec_Gen_loss: 0.0386 - em_2_dec_Gen_loss: 0.0319 - em_concat_loss: 4.3458e-04 - em_fused_loss: 0.0000e+00 - val_loss: 0.1153 - val_em_0_dec_Gen_loss: 0.0378 - val_em_1_dec_Gen_loss: 0.0418 - val_em_2_dec_Gen_loss: 0.0353 - val_em_concat_loss: 3.5666e-04 - val_em_fused_loss: 0.0000e+00
Epoch 3/40
49109/49109 [==============================] - 1525s 31ms/step - loss: 0.1016 - em_0_dec_Gen_loss: 0.0333 - em_1_dec_Gen_lo

In [26]:
a_model.model.load_weights('models/ag_3d_1-100.h5')

In [35]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

file_general_pattern = 'OAS2_0{0:03}_MR{1}_{3}_OAS2_0{0:03}_MR{2}'
dataset_location = 'datasets/OASIS/OASIS2/REG/{}/{}.nii.gz'

mad_results = {}
ssim_results = {}
curr_patch_shape = (32, 32, 32)
step = (16, 16, 16)
for i in range(100, 190) :
    print '{} :'.format(i),
    for j in range(1, 5) :
        for k in range(j+1, 5) :
            ref_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, j, k, 'halfwayto'))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, j, k, 'halfwayto') + '_brain_pve_0')
            mov_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, k, j, 'halfwayto'))
            mov_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                file_general_pattern.format(i, k, j, 'halfwayto') + '_pve_0')
            
            if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
                continue

            ######################################################################################
            volume_init = nib.load(ref_filename).get_data()
            volume_init = volume_init / volume_init.max()
            
            ref_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(ref_prob_filename).get_data()
            
            ref_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_prob_patches = ref_prob_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            ref_train = np.hstack((ref_patches, ref_prob_patches)).astype('float32') ##
            del ref_patches, ref_prob_patches
            ######################################################################################
            volume_init = nib.load(mov_filename).get_data()
            
            mov_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_patches = mov_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(ref_prob_filename).get_data()
            
            mov_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_prob_patches = mov_prob_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            seg_train = np.hstack((mov_prob_patches, mov_patches != 0)).astype('float32') ##
            del mov_patches, mov_prob_patches
            ######################################################################################
            print '{}->{}'.format(j, k),
    
            pred = a_model.model.predict([ref_train, seg_train], verbose=1)[2]
            pred = pred.reshape((-1, ) + curr_patch_shape)
            
            volume = perform_voting(pred, curr_patch_shape, volume_init.shape, step)
    
            volume = volume * nib.load(mov_filename).get_data().max()
            volume = np.multiply(nib.load(mov_filename).get_data() != 0, volume)

            nib.save(nib.Nifti1Image(volume, volume_data.affine),
                     'updated_results/{}_{}_to_{}.nii.gz'.format(i, j, k))
            
            caster = sitk.CastImageFilter()
            caster.SetOutputPixelType(res.GetPixelID())

            res = sitk.ReadImage('updated_results/{}_{}_to_{}.nii.gz'.format(i, j, k))
            orig = caster.Execute(sitk.ReadImage(mov_filename))

            matcher = sitk.HistogramMatchingImageFilter()
            matcher.SetNumberOfHistogramLevels(1024)
            matcher.SetNumberOfMatchPoints(15)
            matched = matcher.Execute(res, orig)

            sitk.WriteImage(matched, 'updated_results/{}_{}_to_{}_cor.nii.gz'.format(i, j, k))

            volume = nib.load('updated_results/{}_{}_to_{}_cor.nii.gz'.format(i, j, k)).get_data()
            ref_vol = np.float64(nib.load(ref_filename).get_data())
            act_vol = np.float64(nib.load(mov_filename).get_data())
            mad_refgen = mad(volume, ref_vol)
            ssim_refgen = ssim(volume, ref_vol)
            mad_movgen = mad(volume, act_vol)
            ssim_movgen = ssim(volume, act_vol)

            mad_results['{} {}->{}'.format(i, j, k)] = mad_movgen
            ssim_results['{} {}->{}'.format(i, j, k)] = ssim_movgen
            print '({}, {}), ({}, {})'.format(mad_refgen, mad_movgen, ssim_refgen, ssim_movgen)

2025/2025 [==============================] - 25s 12ms/step
 0.74413437 -0.00023287923
(0.023008592957, 0.0233841082785), (0.975292146911, 0.970962141009)
2025/2025 [==============================] - 25s 13ms/step
 0.6645979 -0.00022164414
(0.0675926096444, 0.0247629111715), (0.936587679231, 0.96904443348)
2025/2025 [==============================] - 25s 12ms/step
 0.70025605 -0.00022127686
(0.0255815992654, 0.0213702265425), (0.978610968151, 0.974117929116)
2025/2025 [==============================] - 25s 13ms/step
 0.6841287 -0.00047393795
(0.0551399016406, 0.0230007472994), (0.959455831966, 0.959296892184)
2025/2025 [==============================] - 25s 13ms/step
 0.6331741 -0.00019750051
(0.015123734539, 0.0214796851663), (0.976603008739, 0.968552718889)
2025/2025 [==============================] - 25s 13ms/step
 0.6690181 -0.00021175377
(0.0336141671725, 0.0193719428552), (0.970705844898, 0.97364566344)
2025/2025 [==============================] - 25s 13ms/step
 0.7518782 -0.00023

 0.78307706 -0.00028758345
(0.0176746315684, 0.0256323736697), (0.968624385423, 0.95032535357)
2025/2025 [==============================] - 25s 12ms/step
 0.90943205 -0.00024918513
(0.0187712336565, 0.0245845450033), (0.968296725718, 0.950964076257)
2025/2025 [==============================] - 25s 12ms/step
 0.7066629 -0.00024639932
(0.0148194779147, 0.0194861947274), (0.977506104174, 0.976719511918)
2025/2025 [==============================] - 25s 12ms/step
 0.7381734 -0.0004402028
(0.0467750139165, 0.0221731825203), (0.956163938735, 0.96143329786)
2025/2025 [==============================] - 25s 12ms/step
 0.77051276 -0.00046112554
(0.075491943678, 0.0247208247181), (0.928140613073, 0.945136130169)
2025/2025 [==============================] - 25s 12ms/step
 0.748905 -0.00032873364
(0.0466217903778, 0.024912291404), (0.959036169044, 0.955034122606)
2025/2025 [==============================] - 25s 13ms/step
 0.7222054 -0.00023734212
(0.0752641150826, 0.0246862594582), (0.929717220602, 

2025/2025 [==============================] - 25s 13ms/step
 0.723185 -0.00041730635
(0.0157546872951, 0.0201148002427), (0.968178456806, 0.961236599811)
2025/2025 [==============================] - 25s 13ms/step
 0.7804423 -0.00053630344
(0.0193807303629, 0.0206706507938), (0.953717394283, 0.96545440604)
2025/2025 [==============================] - 25s 13ms/step
 0.64237094 -0.0012662997
(0.127922219645, 0.0319901969086), (0.812532063141, 0.955809106561)
2025/2025 [==============================] - 25s 13ms/step
 0.6050552 -0.0010833275
(0.137759444587, 0.0310907282862), (0.784228429054, 0.955407780939)
2025/2025 [==============================] - 25s 13ms/step
 0.65650284 -0.00020977276
(0.0868514523618, 0.0287237276466), (0.935256430642, 0.968932260527)
2025/2025 [==============================] - 25s 12ms/step
 0.8103426 -0.0002261889
(0.0277783172274, 0.024820124422), (0.973288599688, 0.953572681748)
2025/2025 [==============================] - 25s 13ms/step
 0.7336758 -0.000263786